# Vanilla Model with Tensorflow and Keras

This model endeavors to reproduce what we did with fast.ai in TensorFlow so that we can use TensorFlow.js

Source: [here](https://www.tensorflow.org/alpha/tutorials/images/transfer_learning)

## Imports

In [29]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import numpy as np
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

TensorFlow version is  1.13.1


## Data preprocessing
Here we need to get the data we want to use and format it properly.

In [11]:
data_root = tf.keras.utils.get_file(origin='http://data.vision.ee.ethz.ch/cvl/gfanelli/kinect_head_pose_db.tgz', 
                                    fname='head_pose', untar=True)

6014402560/6014398431 [==============================] - 1593s 0us/step


Note that here we have to hard-code in the data_path, because the data_root is not quite right (not really sure why...)

In [0]:
data_path = '/root/.keras/datasets/hpdb/'

In [26]:
cal = np.genfromtxt(data_path + '01/rgb.cal', skip_footer=6); cal

array([[517.679,   0.   , 320.   ],
       [  0.   , 517.679, 240.5  ],
       [  0.   ,   0.   ,   1.   ]])

Now, let us create methods to actually extract the center of the image and try it out

In [0]:
def img2txt_name(f): 
  return data_path + str(f)[:-7] + 'pose.txt'

def convert_biwi(coords):
    c1 = coords[0] * cal[0][0]/coords[2] + cal[0][2]
    c2 = coords[1] * cal[1][1]/coords[2] + cal[1][2]
    return np.array([c2,c1])

def get_ctr(f):
    ctr = np.genfromtxt(img2txt_name(f), skip_header=3)
    return convert_biwi(ctr)
  
def get_scaled_center(f, width, height):
  img = cv2.imread(data_path + f)
  pt = get_ctr(f)
  return np.array([pt[0] * (width / img.shape[0]), pt[1] * (height / img.shape[1])])

In [54]:
fname = '09/frame_00667_rgb.png'
img = cv2.imread(data_path + fname)
print(get_ip_center(fname, img.shape[0], img.shape[1]))

[263.91039223 428.58139299]


## Create Training and Validation Arrays

## Create Image Data Generator with Image Augmentation
We then need to resize the images and create a data generator, to be used for training / validation

For this section, I used an additional source: [here](https://keras.io/preprocessing/image/)

In [0]:
image_width = 160
image_height = 120
batch_size = 32

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow(x_train, y_train
                target_size=(image_width, image_height),  
                batch_size=batch_size,
                # Since we need the output to be an array of ints, we will use sparse
                class_mode='sparse')

# Flow validation images in batches of 32 using test_datagen generator
validation_generator = validation_datagen.flow(x_valid, y_valid
                target_size=(image_width, image_height),
                batch_size=batch_size,
                class_mode='sparse')